# Сиды

In [1]:
import pandas as pd
import numpy as np
import torch
import os
import random
    

seed=42

os.environ['PYTHONHASHSEED']=str(seed)
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)



In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Импорты

In [3]:
from torch import nn
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import timm
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import v2
import tqdm 
from tqdm import tqdm
from PIL import Image
import math
from transformers import AutoTokenizer, AutoModel
import joblib


# TEST MODE

In [4]:
TEST_MODE=True

# Загрузка данных

In [5]:
train=pd.read_csv('/kaggle/input/shopee-product-matching/train.csv')
test=pd.read_csv('/kaggle/input/shopee-product-matching/test.csv')
sample=pd.read_csv('/kaggle/input/shopee-product-matching/sample_submission.csv')

train_img_dir='/kaggle/input/shopee-product-matching/train_images'
test_img_dir='/kaggle/input/shopee-product-matching/test_images'

In [6]:
train.nunique()

posting_id     34250
image          32412
image_phash    28735
title          33117
label_group    11014
dtype: int64

In [7]:
if TEST_MODE:
    #train=train.sample(n=100, random_state=seed).reset_index(drop=True)
    train=train[:32000]
else:
    train=train

# Маппинг

In [8]:
label2id = {lg: i for i, lg in enumerate(sorted(train['label_group'].unique()))}
id2label = {i: lg for lg, i in label2id.items()}

In [9]:
train['class_id']=train['label_group'].map(label2id)

## train_test_split

In [10]:
train_data, eval_data=train_test_split(train, test_size=0.2)
#stratify=train['class_id']

# Датасет

In [11]:
class ShopeeDataset(Dataset):
    def __init__(self, df, img_root, transform, train):
        self.df=df
        self.img_root=img_root
        self.transform=transform
        self.train=train
    def __len__(self):
        return len(self.df)
        
    def __getitem__(self, idx):
        row=self.df.iloc[idx]
        image_path=row['image']
        image_path_all=os.path.join(self.img_root, f'{image_path}')
        image=Image.open(image_path_all).convert('RGB')

        if self.transform is not None:
            image=self.transform(image)
        else:
            image=image
        
        if self.train:
            return({
                'image': image,
                'label': torch.tensor(row['class_id'], dtype=torch.long),
                'posting_id': row['posting_id']
            })
        else:
            return({
                'image': image,
                'posting_id': row['posting_id']
            })
        
        

In [12]:
# === TEXT dataset / loaders ===
class ShopeeTextDataset(Dataset):
    def __init__(self, df, is_train=True):
        self.df = df.reset_index(drop=True)
        self.is_train = is_train
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        item = {
            'title': str(row['title']) if not pd.isna(row['title']) else '',
            'posting_id': row['posting_id'],
        }
        if self.is_train:
            item['label'] = torch.tensor(row['class_id'], dtype=torch.long)
        return item

train_text_ds = ShopeeTextDataset(train_data, is_train=True)
eval_text_ds  = ShopeeTextDataset(eval_data,  is_train=True)

train_text_loader = DataLoader(train_text_ds, batch_size=128, shuffle=True,
                               num_workers=4, pin_memory=True, persistent_workers=True, drop_last=True)
eval_text_loader  = DataLoader(eval_text_ds,  batch_size=256, shuffle=False,
                               num_workers=4, pin_memory=True, persistent_workers=True)


## Аугментации 

In [13]:
transforms_train=v2.Compose([
    v2.Resize((224, 224)), 
    v2.RandomHorizontalFlip(),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

transforms_eval=v2.Compose([
    v2.Resize((224,224)),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transforms_test=v2.Compose([
    v2.Resize((224,224)),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

/usr/local/lib/python3.11/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


## Создание датасетов

In [14]:
train_dataset=ShopeeDataset(train_data, train_img_dir, transform=transforms_train, train=True)
eval_dataset=ShopeeDataset(eval_data,train_img_dir, transform=transforms_eval, train=True)
test_dataset=ShopeeDataset(test, test_img_dir, transform=transforms_test, train=False)

## Даталоадеры

In [15]:
train_dataloader=DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
eval_dataloader=DataLoader(eval_dataset, batch_size=32, shuffle=False, num_workers=4)
test_dataloader=DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

# Кол-во эпох

In [16]:
EPOCHS=5

# Model for image

In [17]:
model=timm.create_model('eca_nfnet_l1', pretrained=True,  num_classes=0).to(device)

model.safetensors:   0%|          | 0.00/166M [00:00<?, ?B/s]

In [18]:
def build_image_embeddings(loader):
    embs, ids, labels_opt = [], [], []
    for b in tqdm(loader, desc="Embed/img"):
        x = b['image'].to(device, non_blocking=True)
        e = model(x)
        e = embedding_head(e)
        e = nn.functional.normalize(e, dim=1)
        embs.append(e.cpu())
        ids.extend(b['posting_id'])
        if 'label' in b:
            labels_opt.extend(b['label'].cpu().numpy().tolist())
    embs = torch.cat(embs, dim=0).numpy().astype('float32')  # (N, 512)
    embs /= (np.linalg.norm(embs, axis=1, keepdims=True) + 1e-8)
    return embs, ids, (np.array(labels_opt) if labels_opt else None)

# Model for text

In [19]:
# === TEXT encoder (XLM-RoBERTa) finetune + ArcFace head ===
TEXT_MODEL_NAME = 'xlm-roberta-base'
text_tokenizer = AutoTokenizer.from_pretrained(TEXT_MODEL_NAME, use_fast=True)
text_model = AutoModel.from_pretrained(TEXT_MODEL_NAME).to(device)  # будет тренироваться
text_feat_dim = text_model.config.hidden_size  # 768

text_embedding_head = nn.Sequential(
    nn.Linear(text_feat_dim, 512, bias=False),
    nn.BatchNorm1d(512)
).to(device)



tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

2025-10-10 09:23:39.633070: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760088219.820729      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760088219.879437      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [20]:
@torch.no_grad()
def build_text_embeddings_for_df(df, batch_size=256, max_len=64):
    """
    Возвращает L2-нормированные эмбеддинги заголовков (N, 768) в float16.
    Порядок совпадает с df.
    """
    titles = df['title'].fillna('').astype(str).tolist()
    embs = []
    for start in tqdm(range(0, len(titles), batch_size), desc="Embed/text"):
        batch = titles[start:start+batch_size]
        tok = text_tokenizer(batch, padding=True, truncation=True,
                             max_length=max_len, return_tensors='pt')
        tok = {k: v.to(device, non_blocking=True) for k, v in tok.items()}
        out = text_model(**tok)
        sent = mean_pooling(out.last_hidden_state, tok['attention_mask'])  # (B, 768)
        sent = nn.functional.normalize(sent, dim=1)
        embs.append(sent.cpu())
    embs = torch.cat(embs, dim=0).numpy().astype('float32')  # (N,768)
    # safety L2 + экономия RAM
    norms = np.linalg.norm(embs, axis=1, keepdims=True) + 1e-8
    return (embs / norms).astype('float16')

def topk_chunked_cos(embs_f16: np.ndarray, K: int, qbs: int = 128):
    """
    OOM-safe top-K по косинусу с помощью torch (без FAISS).
    embs_f16: (N, D) float16, L2-нормированные
    Возвращает sims, idxs: по (N, K) в float32 / int32
    """
    N, D = embs_f16.shape
    device_t = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    db = torch.from_numpy(embs_f16.astype('float32', copy=False)).to(device_t, non_blocking=True)

    K = min(K, N)
    idxs_list, sims_list = [], []
    for start in tqdm(range(0, N, qbs), desc="TopK (torch-chunk)"):
        q = db[start:start+qbs]                 # (qbs, D)
        S = torch.matmul(q, db.T)               # (qbs, N) — косинус
        vals, ids = torch.topk(S, k=K, dim=1, largest=True, sorted=True)
        idxs_list.append(ids.cpu().numpy().astype('int32'))
        sims_list.append(vals.cpu().numpy().astype('float32'))
        del S, vals, ids
        if device_t.type == 'cuda':
            torch.cuda.empty_cache()
    idxs = np.vstack(idxs_list)   # (N, K)
    sims = np.vstack(sims_list)   # (N, K)
    del db
    return sims, idxs


In [21]:
feat_dim = model.num_features 

embedding_head = nn.Sequential(
    nn.Linear(feat_dim, 512, bias=False),
    nn.BatchNorm1d(512)
).to(device)

# ArcFace голова

In [22]:
class ArcFace(nn.Module):
    def __init__(self, in_features, out_features, s=30.0, m=0.5, easy_margin=False):
        super().__init__()
        
        self.in_features=in_features
        self.out_features=out_features
        self.m=m
        self.s=s
        self.easy_margin=easy_margin

        self.weight=nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)
        self.cos_m=math.cos(self.m)
        self.sin_m=math.sin(self.m)

        self.th=math.cos(math.pi-m)
        self.mm=math.sin(math.pi-m)*m
        
        

    def forward(self, embeddings, labels):
        weights=nn.functional.normalize(self.weight)
        cosine=nn.functional.linear(embeddings, weights)
        sine=torch.sqrt(torch.clamp(1.0-cosine**2, min=1e-6))

        phi=cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)

        one_hot = torch.zeros_like(cosine)
        one_hot.scatter_(1, labels.view(-1,1), 1.0)

        
        logits = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        logits = logits * self.s
        return logits

In [23]:
NUM_CLASSES = train['class_id'].nunique()

In [24]:
arcface_head=ArcFace(512, NUM_CLASSES, s=30.0, m=0.5).to(device)

In [25]:
text_arcface_head = ArcFace(512, NUM_CLASSES, s=30.0, m=0.5).to(device)

text_criterion = nn.CrossEntropyLoss(label_smoothing=0.05)

# разные LR для энкодера и головы
from torch.cuda.amp import autocast, GradScaler
txt_lr_backbone = 2e-5
txt_lr_head     = 1e-3

text_optimizer = torch.optim.AdamW([
    {'params': text_model.parameters(), 'lr': txt_lr_backbone, 'weight_decay': 0.01},
    {'params': text_embedding_head.parameters(), 'lr': txt_lr_head, 'weight_decay': 0.01},
    {'params': text_arcface_head.parameters(), 'lr': txt_lr_head, 'weight_decay': 0.00},
])

steps_per_epoch_txt = len(train_text_loader)
EPOCHS_TEXT = 2  # можно 1 в TEST_MODE
text_scheduler = torch.optim.lr_scheduler.OneCycleLR(
    text_optimizer, max_lr=[txt_lr_backbone, txt_lr_head, txt_lr_head],
    epochs=EPOCHS_TEXT, steps_per_epoch=steps_per_epoch_txt,
    pct_start=0.1, div_factor=10, final_div_factor=1e4
)

scaler_txt = GradScaler()


/tmp/ipykernel_36/846053085.py:24: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler_txt = GradScaler()


# Criterion 

In [26]:
criterion=torch.nn.CrossEntropyLoss(label_smoothing=0.05)

    

# Optimizer

In [27]:
optimizer=torch.optim.AdamW(list(model.parameters()) +
    list(embedding_head.parameters()) +
    list(arcface_head.parameters()), lr=3e-4, weight_decay=0.05)

# Scheduler

In [28]:
scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

#  Training loop

In [29]:
for epoch in range(1, EPOCHS+1):
    model.train()
    embedding_head.train()
    arcface_head.train()
    running_loss, num_correct, n=0.0,0,0
    pbar=tqdm(train_dataloader, desc='train', leave=False)
    for step, batch in enumerate(pbar):
        optimizer.zero_grad()
        X=batch['image'].to(device)
        y=batch['label'].to(device)
    
        feats=model(X)
        embeddings = embedding_head(feats)
        embeddings=nn.functional.normalize(embeddings, dim=1)
        logits=arcface_head(embeddings, y)
        loss=criterion(logits, y)
        loss.backward()
        
        
        optimizer.step()
        running_loss+=loss.item()*X.size(0)
        pbar.set_postfix(loss=running_loss/((step+1)*X.size(0)))

    scheduler.step()
# ===== Validation: build embeddings -> cosine-kNN -> global tau -> F1 =====
import numpy as np
def tokenize_titles(titles, max_len=64):
    return text_tokenizer(titles, padding=True, truncation=True, max_length=max_len, return_tensors='pt')

def mean_pooling(last_hidden_state, attention_mask):
    mask = attention_mask.unsqueeze(-1).float()
    summed = (last_hidden_state * mask).sum(dim=1)
    denom = mask.sum(dim=1).clamp(min=1e-6)
    return summed / denom

for epoch in range(1, EPOCHS_TEXT+1):
    text_model.train(); text_embedding_head.train(); text_arcface_head.train()
    pbar = tqdm(train_text_loader, desc=f'train/text e{epoch}', leave=False)
    running = 0.0; nseen = 0
    for batch in pbar:
        titles = batch['title']
        y = batch['label'].to(device, non_blocking=True)

        tok = tokenize_titles(titles)
        tok = {k: v.to(device, non_blocking=True) for k, v in tok.items()}

        text_optimizer.zero_grad(set_to_none=True)
        with autocast():
            out = text_model(**tok)  # last_hidden_state
            sent = mean_pooling(out.last_hidden_state, tok['attention_mask'])     # (B, 768)
            emb  = text_embedding_head(sent)                                      # (B, 512)
            emb  = nn.functional.normalize(emb, dim=1)
            logits = text_arcface_head(emb, y)
            loss = text_criterion(logits, y)

        scaler_txt.scale(loss).backward()
        scaler_txt.step(text_optimizer)
        scaler_txt.update()
        text_scheduler.step()

        bs = y.size(0)
        running += loss.item() * bs
        nseen += bs
        pbar.set_postfix(loss=running / max(1, nseen))

text_model.eval(); text_embedding_head.eval(); text_arcface_head.eval()
@torch.no_grad()
def build_text_embeddings_for_df_ft(df, batch_size=256, max_len=64):
    titles = df['title'].fillna('').astype(str).tolist()
    embs = []
    for i in tqdm(range(0, len(titles), batch_size), desc="Embed/text(ft)"):
        batch = titles[i:i+batch_size]
        tok = tokenize_titles(batch, max_len=max_len)
        tok = {k: v.to(device, non_blocking=True) for k, v in tok.items()}
        out = text_model(**tok)
        sent = mean_pooling(out.last_hidden_state, tok['attention_mask'])  # (B,768)
        e = text_embedding_head(sent)                                      # (B,512)
        e = nn.functional.normalize(e, dim=1)
        embs.append(e.cpu())
    embs = torch.cat(embs, dim=0).numpy().astype('float32')
    embs /= (np.linalg.norm(embs, axis=1, keepdims=True) + 1e-8)
    return embs.astype('float16')  # (N,512) float16

model.eval(); embedding_head.eval(); arcface_head.eval()

@torch.no_grad()
def build_val_embeddings(loader):
    embs, ids, labels = [], [], []
    for b in tqdm(loader, desc="Embed/val"):
        x = b['image'].to(device, non_blocking=True)
        e = model(x)                         # (B, feat_dim)
        e = embedding_head(e)                # (B, 512)
        e = nn.functional.normalize(e, dim=1)
        embs.append(e.cpu())
        ids.extend(b['posting_id'])
        # важно: eval_dataset должен возвращать label (class_id)
        labels.extend(b['label'].cpu().numpy().tolist())
    embs = torch.cat(embs, dim=0).numpy()    # (N, 512)
    labels = np.array(labels)
    return embs, ids, labels

def build_preds(embs, ids, K=50, tau=0.50, mutual=True, cap=50):
    """
    embs: (N, D) L2-нормированные векторы
    ids:  список posting_id в том же порядке
    Возвращает dict: posting_id -> set(predicted_ids) (включая self), с ограничением cap.
    """
    # Косинусная матрица: для L2-нормированных это просто e @ e^T
    S = embs @ embs.T        # (N, N)
    N = S.shape[0]

    # Предвычислим top-K индексы для каждого i (быстро и экономно)
    topk_idx = np.argsort(-S, axis=1)[:, :K]  # индексы соседей по убыванию сходства

    preds = {}
    for i in range(N):
        cand = []
        for j in topk_idx[i]:
            if S[i, j] >= tau:
                if not mutual or i in topk_idx[j]:   # взаимность (mutual-kNN) уменьшает false-merge
                    cand.append(ids[j])
        # обязательно self-match
        if ids[i] not in cand:
            cand = [ids[i]] + cand
        # кап по условию соревна
        preds[ids[i]] = set(cand[:cap])
    return preds

def f1_matches(ids, labels, preds):
    """
    Средний F1 по каждому посту:
      - истина: все posting_id той же class_id
      - предсказание: preds[posting_id]
    """
    # сгруппируем истину: class_id -> set(posting_id)
    truth = {}
    for pid, g in zip(ids, labels):
        truth.setdefault(g, set()).add(pid)

    f1s = []
    for pid, g in zip(ids, labels):
        T = truth[g]
        P = preds[pid]
        inter = len(T & P)
        denom = len(T) + len(P)
        f1s.append( 2*inter/denom if denom > 0 else 0.0 )
    return float(np.mean(f1s))

# 1) эмбеддинги на валидации
val_embs, val_ids, val_labels = build_val_embeddings(eval_dataloader)
# === TEXT embeddings на валидации (в порядке eval_dataset.df) ===
val_text_embs = build_text_embeddings_for_df_ft(eval_dataset.df, batch_size=256, max_len=64)  # (Nv,768) float16

# === top-K для image и text (шире cap, потом обрежем) ===
KQ = 100  # ширина кандидатов перед обрезкой до 50
sims_img, idxs_img = topk_chunked_cos(val_embs.astype('float16'), K=KQ, qbs=128)     # (Nv,KQ)
sims_txt, idxs_txt = topk_chunked_cos(val_text_embs,              K=KQ, qbs=256)     # (Nv,KQ)

# === REPLACE this function with the one below ===
def build_preds_fused_mutual(ids, idxs_img, sims_img, idxs_txt, sims_txt,
                             alpha=0.7, tau=0.50, K_cap=50):
    """
    ids: список posting_id длины N
    idxs_*: (N, KQ) индексы кандидатов
    sims_*: (N, KQ) косинусы [-1,1]
    alpha: вес image (0..1), (1-alpha) — вес text
    tau: порог на fused-скор (после приведения в [0,1])
    K_cap: ограничение размера группы
    """
    N = len(ids)
    # объединённые кандидаты для каждого i
    cand_sets = [ set(idxs_img[i]).union(set(idxs_txt[i])) for i in range(N) ]

    out = {}
    for i in range(N):
        # карты скорингов для быстрого доступа
        map_img = {int(j): float(s) for j, s in zip(idxs_img[i], sims_img[i])}
        map_txt = {int(j): float(s) for j, s in zip(idxs_txt[i], sims_txt[i])}

        fused = []
        for j in cand_sets[i]:
            # приводим косинусы в [0,1] и смешиваем
            si = (map_img.get(j, -1.0) + 1.0) / 2.0
            st = (map_txt.get(j, -1.0) + 1.0) / 2.0
            s  = alpha * si + (1.0 - alpha) * st
            # взаимность на объединённых кандидатах
            if s >= tau and (i in cand_sets[j]):
                fused.append((j, s))

        fused.sort(key=lambda x: -x[1])
        keep = [ids[j] for j, _ in fused]
        if ids[i] not in keep:
            keep = [ids[i]] + keep
        out[ids[i]] = set(keep[:K_cap])
    return out


# Поиск (alpha, tau) по сетке
# --- grid search for fusion params ---
alphas = np.linspace(0.4, 0.9, 6)     # 0.40..0.90
taus   = np.linspace(0.20, 0.80, 31)  # 0.20..0.80

best_fusion = {'f1': -1.0, 'alpha': None, 'tau': None}
for a in alphas:
    for t in taus:
        preds = build_preds_fused_mutual(
            val_ids, idxs_img, sims_img, idxs_txt, sims_txt,
            alpha=float(a), tau=float(t), K_cap=50
        )
        f1 = f1_matches(val_ids, val_labels, preds)
        if f1 > best_fusion['f1']:
            best_fusion = {'f1': float(f1), 'alpha': float(a), 'tau': float(t)}

print(f"[VAL FUSION] Best F1={best_fusion['f1']:.4f} at alpha={best_fusion['alpha']:.2f}, tau={best_fusion['tau']:.2f}")


# (на всякий случай) L2-норм ещё раз, если вызывалась неявно
preds_best_fused = build_preds_fused_mutual(
    val_ids, idxs_img, sims_img, idxs_txt, sims_txt,
    alpha=best_fusion['alpha'], tau=best_fusion['tau'], K_cap=50
)
avg_group_size = np.mean([len(v) for v in preds_best_fused.values()])
print(f"[VAL FUSION] Avg predicted group size: {avg_group_size:.2f}")



# (необязательно) Быстрый sanity-check: средний размер предсказанных групп
# === Save checkpoint(s) ===
SAVE_DIR = '/kaggle/working'
os.makedirs(SAVE_DIR, exist_ok=True)

ckpt = {
    'backbone_name': 'eca_nfnet_l1',          # FIX: было 'resnet50'
    'feat_dim': model.num_features,
    'emb_dim': 512,
    'num_classes': NUM_CLASSES,
    'arcface_cfg': {'s': 30.0, 'm': 0.5, 'easy_margin': False},
    'state_dict': {
        'backbone': model.state_dict(),
        'embedding_head': embedding_head.state_dict(),
        'arcface_head': arcface_head.state_dict(),
    },
    'label2id': label2id,
    'fusion_alpha': float(best_fusion['alpha']),
    'fusion_tau': float(best_fusion['tau']),
    'val_f1_fused': float(best_fusion['f1']),
    'epoch': EPOCHS,
}

torch.save(ckpt, os.path.join(SAVE_DIR, 'arcface_eca_nfnet_l1_shopee_ckpt.pth'))
print('[SAVE] Full checkpoint ->', os.path.join(SAVE_DIR, 'arcface_eca_nfnet_l1_shopee_ckpt.pth'))

# Лёгкий пакет для инференса (нужны веса и параметры фьюжна)
embed_pkg = {
    'backbone_name': 'eca_nfnet_l1',
    'feat_dim': model.num_features,
    'emb_dim': 512,
    'state_dict': {
        'backbone': model.state_dict(),
        'embedding_head': embedding_head.state_dict(),
    },
    'fusion_alpha': float(best_fusion['alpha']),
    'fusion_tau': float(best_fusion['tau']),
}
torch.save(embed_pkg, os.path.join(SAVE_DIR, 'embedding_extractor.pth'))
print('[SAVE] Embedding extractor ->', os.path.join(SAVE_DIR, 'embedding_extractor.pth'))


train/text e1:   0%|          | 0/200 [00:00<?, ?it/s]             /tmp/ipykernel_36/3844142818.py:48: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Embed/val:   0%|          | 0/200 [00:00<?, ?it/s]                         huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The curr

[VAL FUSION] Best F1=0.9166 at alpha=0.90, tau=0.74
[VAL FUSION] Avg predicted group size: 1.78
[SAVE] Full checkpoint -> /kaggle/working/arcface_eca_nfnet_l1_shopee_ckpt.pth
[SAVE] Embedding extractor -> /kaggle/working/embedding_extractor.pth


In [30]:
# --- SAVE full ckpt (image + text) ---
ckpt = {
    'backbone_name_img': 'eca_nfnet_l1',
    'backbone_name_txt': TEXT_MODEL_NAME,
    'img_feat_dim': model.num_features,
    'txt_feat_dim': text_feat_dim,  # 768
    'emb_dim': 512,
    'num_classes': NUM_CLASSES,
    'arcface_cfg': {'s': 30.0, 'm': 0.5, 'easy_margin': False},
    'state_dict': {
        'img_backbone': model.state_dict(),
        'img_embed_head': embedding_head.state_dict(),
        'img_arcface_head': arcface_head.state_dict(),
        'txt_backbone': text_model.state_dict(),
        'txt_embed_head': text_embedding_head.state_dict(),
        'txt_arcface_head': text_arcface_head.state_dict(),
    },
    'label2id': label2id,
    'fusion_alpha': float(best_fusion['alpha']),
    'fusion_tau': float(best_fusion['tau']),
    'val_f1_fused': float(best_fusion['f1']),
    'epoch_img': EPOCHS,
    'epoch_txt': EPOCHS_TEXT,
}
torch.save(ckpt, os.path.join(SAVE_DIR, 'imgtxt_arcface_shopee_ckpt.pth'))

# --- SAVE light embed extractor (для теста) ---
embed_pkg = {
    'backbone_name_img': 'eca_nfnet_l1',
    'backbone_name_txt': TEXT_MODEL_NAME,
    'emb_dim': 512,
    'state_dict': {
        'img_backbone': model.state_dict(),
        'img_embed_head': embedding_head.state_dict(),
        'txt_backbone': text_model.state_dict(),
        'txt_embed_head': text_embedding_head.state_dict(),
    },
    'fusion_alpha': float(best_fusion['alpha']),
    'fusion_tau': float(best_fusion['tau']),
}
torch.save(embed_pkg, os.path.join(SAVE_DIR, 'embedding_extractor_mix.pth'))


# Submission

In [31]:
#need inference notebook